In [1]:
import collections
import os
import multiprocessing
import sys

import shap
import pandas as pd
import numpy as np
import lightgbm as lgb
import sklearn.metrics
import ml_metrics
import tqdm
from gensim.models import Word2Vec

In [2]:
# Reproducibility parameters
SEED = 281989

# Explainability parameters
EXPLAIN_FROM_SAMPLES = 1000

# LightGBM parameters
LIGHTGBM_LEARNING_RATE = 0.2
LIGHTGBM_NUM_BOOST_ROUND = 50

# Word2Vec parameters
WORD2VEC_EMBEDDING_SIZE = 32
WORD2VEC_EPOCHS = 10

# Evaluation parameters
CROSS_VALIDATION_K = 5
MAP_K = 10
EVAL_USERS_LIMIT = 100

In [3]:
# Data description: https://www.kaggle.com/c/kkbox-music-recommendation-challenge/data
data_dir = 'data'

In [4]:
train_columns = {
  'msno': 'category',
  'song_id': 'category',
  'source_system_tab': 'category',
  'source_screen_name': 'category',
  'source_type': 'category',
  'target': np.float,
}
train_df_raw = pd.read_csv(os.path.join(data_dir, 'train.csv'), dtype=train_columns)
print(train_df_raw.dtypes)
train_df_raw.head()

msno                  category
song_id               category
source_system_tab     category
source_screen_name    category
source_type           category
target                 float64
dtype: object


,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1.0


In [5]:
test_columns = {
  'id': np.int,
  'msno': 'category',
  'song_id': 'category',
  'source_system_tab': 'category',
  'source_screen_name': 'category',
  'source_type': 'category',
}
test_df_raw = pd.read_csv(os.path.join(data_dir, 'test.csv'), dtype=test_columns)
print(test_df_raw.dtypes)
test_df_raw.head()

id                       int64
msno                  category
song_id               category
source_system_tab     category
source_screen_name    category
source_type           category
dtype: object


,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


In [6]:
songs_columns = {
  'song_id': 'category',
  'song_length': np.int,
  'genre_ids': 'category',
  'artist_name': 'category',
  'composer': 'category',
  'lyricist': 'category',
  'language': 'category',
}
songs_df_raw = pd.read_csv(os.path.join(data_dir, 'songs.csv'), dtype=songs_columns)
print(songs_df_raw.dtypes)
songs_df_raw.head()

song_id        category
song_length       int64
genre_ids      category
artist_name    category
composer       category
lyricist       category
language       category
dtype: object


,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [7]:
members_columns = {
  'msno': 'category',
  'city': 'category',
  'bd': np.int,
  'gender': 'category',
  'registered_via': 'category',
  'registration_init_time': str,
  'expiration_date': str,
}
members_df_raw = pd.read_csv(os.path.join(data_dir, 'members.csv'), dtype=members_columns)
print(members_df_raw.dtypes)
members_df_raw.head()

msno                      category
city                      category
bd                           int64
gender                    category
registered_via            category
registration_init_time      object
expiration_date             object
dtype: object


,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [8]:
song_extra_info_columns = {
  'song_id': 'category',
  'name': 'category',
  'isrc': 'category',
}
song_extra_info_df_raw = pd.read_csv(os.path.join(data_dir, 'song_extra_info.csv'), dtype=song_extra_info_columns)
print(song_extra_info_df_raw.dtypes)
song_extra_info_df_raw.head()

song_id    category
name       category
isrc       category
dtype: object


,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


In [9]:
song_columns_to_use = ['song_id', 'artist_name', 'genre_ids', 'song_length', 'language']
train_df = train_df_raw.merge(songs_df_raw[song_columns_to_use], on='song_id', how='left')
test_df = test_df_raw.merge(songs_df_raw[song_columns_to_use], on='song_id', how='left')

In [10]:
def parse_yearmonthday(date):
  parts = date[0:4], date[4:6], date[6:8]
  return list(map(int, parts))

In [11]:
members_df = members_df_raw.copy()

In [12]:
reg_init_label = 'registration_init_time'
members_df['registration_year'] = members_df[reg_init_label].apply(lambda date: parse_yearmonthday(date)[0])
members_df['registration_month'] = members_df[reg_init_label].apply(lambda date: parse_yearmonthday(date)[1])
members_df['registration_day'] = members_df[reg_init_label].apply(lambda date: parse_yearmonthday(date)[2])
members_df = members_df.drop([reg_init_label], axis=1)

In [13]:
exp_label = 'expiration_date'
members_df['expiration_year'] = members_df[exp_label].apply(lambda date: parse_yearmonthday(date)[0])
members_df['expiration_month'] = members_df[exp_label].apply(lambda date: parse_yearmonthday(date)[1])
members_df['expiration_day'] = members_df[exp_label].apply(lambda date: parse_yearmonthday(date)[2])
members_df = members_df.drop([exp_label], axis=1)

In [14]:
train_df = train_df.merge(members_df, on='msno', how='left')
test_df = test_df.merge(members_df, on='msno', how='left')

In [15]:
def isrc_to_year(isrc):
  if type(isrc) == str:
    if int(isrc[5:7]) > 17:
      return 1900 + int(isrc[5:7])
    else:
      return 2000 + int(isrc[5:7])
  else:
    return np.nan

song_extra_info_df = song_extra_info_df_raw.copy()
song_extra_info_df['song_year'] = song_extra_info_df['isrc'].apply(isrc_to_year)
song_extra_info_df.drop(['isrc', 'name'], axis=1, inplace=True)

In [16]:
train_df = train_df.merge(song_extra_info_df, on='song_id', how='left')
test_df = test_df.merge(song_extra_info_df, on='song_id', how='left')

In [17]:
for col in train_df.columns:
  if train_df[col].dtype == object:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

In [18]:
def train_lightgbm(X_train, y_train):
  D_train = lgb.Dataset(X_train, y_train)
  watchlist = [D_train]

  params = {
    'learning_rate': LIGHTGBM_LEARNING_RATE,
    'objective': 'binary',
    'max_depth': 8,
    'num_leaves': 2**8,
    'verbosity': 0,
    'metric': 'auc',
  }

  print('Training the model')
  model = lgb.train(
    params,
    train_set=D_train,
    num_boost_round=LIGHTGBM_NUM_BOOST_ROUND,
    valid_sets=watchlist,
    verbose_eval=10)
  return model

In [19]:
def lightgbm_rank_by_user(model, user_id, songs_df):
  song_ids = songs_df.song_id.tolist()
  preds = model.predict(songs_df)
  indexes = list(range(len(song_ids)))
  indexes.sort(key=lambda index: preds[index], reverse=True)
  return [song_ids[index] for index in indexes]

In [20]:
ModelEvaluation = collections.namedtuple('ModelEvaluation', ['mapk'])

def evaluate_model_ranking_fn(ranking_fn, X_val, y_val) -> ModelEvaluation:
  print('Evaluating the model')
  sys.stdout.flush()
  val_user_ids = X_val.msno.unique().tolist()
  X_val_target_1 = X_val[y_val == 1.0]
  apks = []
  for user_id in tqdm.tqdm(val_user_ids[:EVAL_USERS_LIMIT], file=sys.stdout):
    songs_df = X_val[X_val.msno == user_id]
    predicted = ranking_fn(user_id, songs_df)
    actual = X_val_target_1[X_val_target_1.msno == user_id].song_id.tolist()
    apk = ml_metrics.apk(actual, predicted, MAP_K)
    apks.append(apk)
  sys.stdout.flush()
  evaluation = ModelEvaluation(mapk=np.mean(apks))
  print(evaluation)
  return evaluation

In [21]:
def k_fold_cross_validation(train_df, k, train_model_fn, preprocess_val_fn, build_ranking_fn):
  indexes = np.array(train_df.index)
  np.random.seed(SEED)
  np.random.shuffle(indexes)
  X_train_splits = np.array_split(indexes, k)
  
  evaluations = []
  for split_index, val_indexes in enumerate(X_train_splits):
    print('*' * 100)
    print(f'Taking split number {split_index} as the validation set...')
    train_indexes = np.setdiff1d(indexes, val_indexes)
    print('Train indexes:', len(train_indexes))
    print('Val indexes:', len(val_indexes))

    X_train = train_df.iloc[train_indexes].drop(['target'], axis=1)
    y_train = train_df.iloc[train_indexes]['target'].values
    X_val = train_df.iloc[val_indexes].drop(['target'], axis=1)
    y_val = train_df.iloc[val_indexes]['target'].values
    
    model = train_model_fn(X_train, y_train)
    if preprocess_val_fn:
      X_val = preprocess_val_fn(model, X_val)
    evaluation = evaluate_model_ranking_fn(build_ranking_fn(model), X_val, y_val)
    evaluations.append(evaluation)
  return evaluations

In [22]:
evaluations = k_fold_cross_validation(
  train_df,
  k=CROSS_VALIDATION_K,
  train_model_fn=train_lightgbm,
  preprocess_val_fn=None,
  build_ranking_fn=lambda model: lambda user_id, songs_df: lightgbm_rank_by_user(model, user_id, songs_df))
mapks = list(map(lambda evaluation: evaluation.mapk, evaluations))
np.mean(mapks)

****************************************************************************************************
Taking split number 0 as the validation set...
Train indexes: 5901934
Val indexes: 1475484
Training the model
[10]	training's auc: 0.719914
[20]	training's auc: 0.735223
[30]	training's auc: 0.741772
[40]	training's auc: 0.746365
[50]	training's auc: 0.7503
Evaluating the model
100%|██████████| 100/100 [03:50<00:00,  2.30s/it]
ModelEvaluation(mapk=0.6497512959813556)
****************************************************************************************************
Taking split number 1 as the validation set...
Train indexes: 5901934
Val indexes: 1475484
Training the model
[10]	training's auc: 0.719796
[20]	training's auc: 0.732967
[30]	training's auc: 0.739175
[40]	training's auc: 0.744729
[50]	training's auc: 0.74939
Evaluating the model
100%|██████████| 100/100 [03:51<00:00,  2.32s/it]
ModelEvaluation(mapk=0.6371305555555555)
*********************************************************

0.6470269800327539

In [23]:
X_train = train_df.drop(['target'], axis=1)
y_train = train_df['target'].values
model = train_lightgbm(X_train, y_train)
evaluation = evaluate_model_ranking_fn(
  lambda user_id, songs_df: lightgbm_rank_by_user(model, user_id, songs_df),
  X_train,
  y_train)

Training the model
[10]	training's auc: 0.719427
[20]	training's auc: 0.732026
[30]	training's auc: 0.739266
[40]	training's auc: 0.743502
[50]	training's auc: 0.750068
Evaluating the model
100%|██████████| 100/100 [03:54<00:00,  2.34s/it]
ModelEvaluation(mapk=0.8176242063492063)


In [24]:
np.random.seed(SEED)
explain_indexes = np.random.choice(X_train.index, EXPLAIN_FROM_SAMPLES, replace=False)
X_explain = X_train.iloc[explain_indexes]

In [25]:
explainer = shap.TreeExplainer(model, feature_perturbation='tree_path_dependent')
shap_values = explainer.shap_values(X_explain)

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [26]:
index = 0
shap.initjs()
shap.force_plot(
  explainer.expected_value[0],
  shap_values[0][index,:],
  X_explain.iloc[index,:])

In [27]:
X_test = test_df.drop(['id'], axis=1)
test_ids = test_df['id'].values

p_test = model.predict(X_test)

submission_df = pd.DataFrame()
submission_df['id'] = test_ids
submission_df['target'] = p_test
submission_df.to_csv('lightgbm_submission_df.csv', index=False, float_format='%.6f')

# Nonclassical embeddings: Word2Vec

In [28]:
user_embeddings = None

def rebuild_user_embeddings(wv, X_train, y_train):
  global user_embeddings
  user_ids = X_train.msno.unique().tolist()
  user_embeddings = collections.defaultdict(lambda: np.zeros(WORD2VEC_EMBEDDING_SIZE))
  user_counts = collections.defaultdict(int)
  print('Building user embeddings')
  for (index, row), target in tqdm.tqdm(zip(X_train.iterrows(), y_train), file=sys.stdout):
    if target != 1.0:
      continue
    user_id = row['msno']
    song_id = row['song_id']
    if song_id in wv:
      user_embeddings[user_id] += wv[song_id]
      user_counts[user_id] += 1
  for user_id in user_embeddings.keys():
    user_embeddings[user_id] /= user_counts[user_id]

def train_word2vec(X_train, y_train):
  user_ids = X_train.msno.unique().tolist()
  print('Users:', len(user_ids))
  
  print('Building sessions')
  sessions = []
  for user_id in user_ids:
    sessions.append(X_train[X_train['msno'] == user_id]['song_id'].tolist())

  word2vec = Word2Vec(
    window=10, size=WORD2VEC_EMBEDDING_SIZE, sg=1, hs=0, negative=10, alpha=0.03, min_alpha=0.0007,
    workers=multiprocessing.cpu_count(), seed=SEED)
  
  print('Building vocabulary')
  word2vec.build_vocab(sessions, progress_per=1000)
  
  print('Training word2vec')
  word2vec.train(sessions, total_examples=word2vec.corpus_count, epochs=WORD2VEC_EPOCHS, report_delay=1)
  print(word2vec)
  
  word2vec.init_sims(replace=True)
  rebuild_user_embeddings(word2vec.wv, X_train, y_train)
  return word2vec

In [29]:
word2vec = train_word2vec(X_train, y_train)

Users: 30755
Building sessions
Building vocabulary
Training word2vec
Word2Vec(vocab=101648, size=32, alpha=0.03)
Building user embeddings
7377418it [06:48, 18076.52it/s]


## Word2Vec similars

In [30]:
def similar_song_ids(song_id, n=10):
  song_embedding = word2vec.wv[song_id]
  similars = word2vec.wv.similar_by_vector(song_embedding, topn=n+1)[1:]
  return similars

In [31]:
song_id = 'CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E='
print(song_extra_info_df_raw[song_extra_info_df_raw.song_id == song_id])

                                             song_id name          isrc
200812  CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=   焚情  TWB531410010


In [32]:
for similar_id, similarity in similar_song_ids(song_id):
  print(song_extra_info_df_raw[song_extra_info_df_raw.song_id == similar_id])
  print('Similarity is', similarity)

                                            song_id   name          isrc
81585  1CLJhlrmGXmFNrFwH2L6fOMcp4bwYQMJPBbUEitZd2k=  愛你的宿命  TWB531410001
Similarity is 0.8702131509780884
                                             song_id  name          isrc
412187  sGjtw+Dl6XPOd4inrRiEzv1pCeQN9Wtml6egb0CZw2I=  上海姑娘  TWB530610008
Similarity is 0.8310494422912598
                                             song_id name          isrc
152575  pbjE7VuJhc7fiECF0GEp6kNvopMAAskldFR2zS5ThG0=  情歌手  TWA130100804
Similarity is 0.8216755390167236
                                             song_id name          isrc
131161  r5q9D1UeT2JNPFHnVkAU5PeL/8v8ord5CJDI9JQ3US0=   柔軟  TWB531410003
Similarity is 0.8211809992790222
                                             song_id name          isrc
179216  i2r2H9csf9O8WcqfBSluK/Q8/rPFgd+/Ba19s7AbRhY=   自從  TWA130200301
Similarity is 0.8155577778816223
                                             song_id  \
300076  iq0pljzLSoNxy9+hPpRob3Iu2oOB+27B/OELxjh6WWg=   

## Word2Vec as a recommender system

In [33]:
def get_user_word2vec_embedding(user_id):
  global user_embeddings
  return user_embeddings[user_id]

In [34]:
sample_user_id = 'FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg='
sample_user_embedding = get_user_word2vec_embedding(sample_user_id)
sample_user_embedding

array([-0.18062335, -0.09155007,  0.12431272,  0.02828265,  0.02962735,
        0.08878952,  0.12603288,  0.08118049, -0.13190042, -0.03563733,
        0.02485227, -0.07927597, -0.01223744,  0.12591707,  0.09178266,
        0.0182774 , -0.17914344, -0.14955795,  0.13865888, -0.05123768,
        0.22304841,  0.19278718,  0.15587596, -0.01156536,  0.03680101,
        0.01595459,  0.05715868,  0.20247165, -0.05994776,  0.20214146,
        0.06441861,  0.08595817])

In [35]:
def word2vec_similarity(user_embedding, song_embedding):
  user_embedding_norm = np.linalg.norm(user_embedding)
  song_embedding_norm = np.linalg.norm(song_embedding)
  if user_embedding_norm < 1e-9 or song_embedding_norm < 1e-9:
    similarity = np.nan
  else:
    similarity = np.dot(user_embedding, song_embedding) / (user_embedding_norm * song_embedding_norm)
  return similarity

In [36]:
def word2vec_rank_by_user(wv, user_id, songs_df):
  song_ids = songs_df.song_id.tolist()
  user_embedding = get_user_word2vec_embedding(user_id)
  similarities = []
  for song_id in song_ids:
    song_embedding = wv[song_id] if song_id in wv else np.zeros(WORD2VEC_EMBEDDING_SIZE)
    similarity = word2vec_similarity(user_embedding, song_embedding)
    similarities.append(similarity)
  
  indexes = list(range(len(song_ids)))
  indexes.sort(key=lambda index: similarities[index], reverse=True)
  # TODO(niksaz): Remove this debug output.
  print([similarities[index] for index in indexes])
  return [song_ids[index] for index in indexes]

In [37]:
evaluation = evaluate_model_ranking_fn(
  lambda user_id, songs_df: word2vec_rank_by_user(word2vec.wv, user_id, songs_df),
  X_train,
  y_train)

Evaluating the model
  0%|          | 0/100 [00:00<?, ?it/s][0.8403348488219571, 0.8372094453374771, 0.8316751162553673, 0.8295715352725928, 0.8259724807174932, 0.8155157913147981, 0.812831290460857, 0.8116178927998291, 0.809495891005319, 0.8068232243578378, 0.8023507164400442, 0.8015980026252743, 0.7827639797190786, 0.7825045840894533, 0.7775649665318057, 0.7761423259896509, 0.7746737301389255, 0.7717582445327662, 0.7712479758647394, 0.7684897337245494, 0.7683661812086965, 0.7674490321557714, 0.7652463036159205, 0.7554248741930392, 0.7533015519529948, 0.748115243683336, 0.7440059610158608, 0.7428272502163711, 0.7422521652290875, 0.7415247932432506, 0.7414856326026671, 0.7405499060990787, 0.7405437878730002, 0.7385547450139298, 0.7358625664945478, 0.7336399797626013, 0.7333623790604524, 0.7312754492807341, 0.7263104650553815, 0.7259949600506548, 0.7255922803858273, 0.7253038225037337, 0.7247457229776296, 0.7209606328831706, 0.7206786319975417, 0.7200410823144509, 0.719833115043763, 0.7

  4%|▍         | 4/100 [00:00<00:02, 39.17it/s][0.8773800507726064, 0.8625584734665844, 0.8612113940937318, 0.8585526816969594, 0.8581237603871916, 0.8580178502473581, 0.8538181657044386, 0.8464969382880606, 0.8464411872289541, 0.8427745923719147, 0.8426715198911142, 0.8395964226335022, 0.8395816414444054, 0.8372909711982196, 0.8349661346501464, 0.8299317283549418, 0.8223294723023717, 0.821740690951784, 0.8146616303374136, 0.8083514238637175, 0.7975544479170583, 0.7925142975320727, 0.7877156936238714, 0.7845947581819716, 0.7796304511299529, 0.7767874669518726, 0.7716682179283094, 0.7684376318219593, 0.7672214017783363, 0.7628053246424574, 0.7600237027320049, 0.7588705757183563, 0.757267335404591, 0.7412263387700473, 0.7267623536265959, 0.7244925398599394, 0.7244077092162178, 0.7234395642840644, 0.7183524938686675, 0.7178534297177356, 0.7157182211411274, 0.714827001788841, 0.7116912444282409, 0.7087130858307148, 0.7085012670665816, 0.7069034422593712, 0.6962884089004909, 0.6902438160907

[nan, nan, 0.8156188629271206, 0.7955004869270312, 0.7950367148016867, 0.7945945361335096, 0.7817799899216824, 0.7806715130292804, 0.7669861267561506, 0.7650331788326038, 0.7611884076571576, 0.7503129628523278, 0.7496164439755253, 0.7472013276827957, 0.7460743455515352, 0.7415022489831606, 0.7371557739331054, 0.7370378200392397, 0.7192446317783682, 0.7182404703850864, 0.7178314136744721, 0.7152252811462224, 0.713785177882054, 0.7135686002960365, 0.7133787636223484, 0.7066818725917315, 0.7008724627505707, 0.6990742199278059, 0.6963800575794319, 0.6948678412787723, 0.694574955857034, 0.6934305179637948, 0.6879288124838923, 0.6874558459809176, 0.6854946330562227, 0.685302007652638, 0.6845422705360029, 0.6833565835732688, 0.6820084175891801, 0.6818789965168616, 0.6793618244827917, 0.6764581036315144, 0.6751068433014434, 0.6722186889527678, 0.6720750092682808, 0.6686068228738964, 0.6652754106573013, 0.6633058958002436, 0.658624127980045, 0.6567538457735184, 0.6521533052682275, 0.65123711071

 13%|█▎        | 13/100 [00:00<00:01, 47.07it/s][nan, 0.9131458081899093, 0.9076236484777852, 0.9033999681086741, 0.9032171859785885, 0.9030849258902244, 0.8988544830820346, 0.897249685701704, 0.8966442530478553, 0.8944199167778946, 0.8900896405885614, 0.8880350737240236, 0.887955478000082, 0.8866930778627982, 0.885813988694327, 0.8836189685181178, 0.883242639705943, 0.881882820069748, 0.8794524991788546, 0.8752034662535086, 0.8740536265309745, 0.873471960703862, 0.8715883856010254, 0.8705394942947058, 0.8703711715843564, 0.869811086076592, 0.8688757332826906, 0.8687632599087841, 0.8675783893566328, 0.8658008886888324, 0.8648512488308956, 0.8643824664356563, 0.8640086450870792, 0.8639897525475265, 0.8621633047202735, 0.8612944218181178, 0.8611856288053185, 0.8609423655756817, 0.8603524225125737, 0.8564053341761181, 0.8563122749418219, 0.8562537506334607, 0.8561661061861644, 0.8554478122167476, 0.8549490902474624, 0.8536136747157799, 0.8532918071875812, 0.8529164394227774, 0.85251716645

[0.8644017434415426, 0.8584821304620902, 0.8568560330439459, 0.8561506294553315, 0.8505220647213075, 0.8504013476654458, 0.8499585616191829, 0.8488020277758508, 0.8458673538984417, 0.8431206236155322, 0.8417737312617415, 0.841293561013468, 0.8396605550081284, 0.8394892090724002, 0.8388889512153167, 0.8387343095866475, 0.8379641107300805, 0.8368498413163631, 0.8359424012520722, 0.8325842648083911, 0.8321343887583158, 0.8285819421800951, 0.8283888159574216, 0.8283264308609493, 0.8266607280615011, 0.8248742345597652, 0.8224393384172061, 0.8221613122217472, 0.8219484935928217, 0.8201451923509582, 0.8197983156494247, 0.8191096132763221, 0.8182992712487566, 0.8160255104187563, 0.8147671370933279, 0.813806446032904, 0.8099141993754215, 0.8098661916327765, 0.80945195326376, 0.8094262781066129, 0.8090486711408541, 0.8090281711239609, 0.8078402315816166, 0.8061897450721155, 0.8044957737885211, 0.8044924240665752, 0.8042295663416207, 0.8035498116068757, 0.8034721866551258, 0.8032402919867205, 0.8

...

[0.9311744152192173, 0.9134475773070263, 0.8970218313335295, 0.8936648878214074, 0.8908117332290233, 0.8865073917795909, 0.8764322898200796, 0.8748847149126356, 0.8739176217694157, 0.8695817908115168, 0.8691863696967437, 0.8685913225592216, 0.8678803089386165, 0.8656437073454804, 0.8656026274217087, 0.8647532300133346, 0.8647020044323825, 0.8638715648520975, 0.8633510510235743, 0.8627084262464106, 0.8624785603551612, 0.8624022423987713, 0.860796969324513, 0.8598805543262594, 0.8535759549612544, 0.851998973229728, 0.8494198136685327, 0.8456406003860184, 0.8439456684688591, 0.8418090199889087, 0.841229643955316, 0.8392807673338761, 0.8342823236407808, 0.8342585901510088, 0.8320405658781056, 0.8292760875211882, 0.8277128976905025, 0.8238562209239095, 0.8236836022119189, 0.8195986018611481, 0.8181036914302316, 0.8174622050912901, 0.8154581921254743, 0.8127335601622288, 0.805535063193944, 0.7972508618941915, 0.7951130020422568, 0.7946371319778623, 0.7946290815639945, 0.791048298465933, 0.78

[0.8582752603518758, 0.8224916572287002, 0.8203805672654892, 0.8196126968966873, 0.8194845780939537, 0.8184850054222778, 0.8175928787544071, 0.8161638626301816, 0.8152985315048122, 0.8150196043953676, 0.8145488935424111, 0.8141058582575238, 0.8128948946877937, 0.8126758321635561, 0.8118183093525664, 0.8107871416526934, 0.8019035898723955, 0.8000795121596774, 0.8000512965955636, 0.797923078816562, 0.7942821145878447, 0.7893430725889553, 0.7832689693535408, 0.7815273137069104, 0.7800603285742662, 0.7792426683663337, 0.7782398853708077, 0.7772577826494957, 0.776977346240097, 0.773297622682094, 0.7718040672686026, 0.7715113990505869, 0.7696153644145232, 0.7695700700839495, 0.7687927091899406, 0.7683415456607604, 0.7656193774789847, 0.7650404076860358, 0.7612249215531387, 0.7594673394913967, 0.7580228319290994, 0.755043841339365, 0.7535626271908552, 0.75316266807235, 0.751299882954583, 0.751230613891708, 0.7503469823691562, 0.7500585474143497, 0.7431928590501617, 0.741081784316504, 0.734752

[0.9367153756498957, 0.934772927195238, 0.9325877155908866, 0.9288704069565016, 0.9233840359005241, 0.9211960682405647, 0.9203378069702955, 0.9195661579120639, 0.917766318493343, 0.9151770421185018, 0.9148466321329501, 0.9146516757686382, 0.914620277071798, 0.9142856204955979, 0.9133855219225282, 0.9125935770878993, 0.9125686314883766, 0.9116443337157337, 0.9114133309181359, 0.9109075756621573, 0.9096768769037304, 0.9089705001675126, 0.9089370950066229, 0.9088555081688604, 0.9076507633102251, 0.9074972888287569, 0.9070540796441068, 0.9070084706547735, 0.9057978428845278, 0.9050651528143986, 0.90475219217315, 0.9045652304269647, 0.9004649744349198, 0.8993849358150845, 0.8987044533659501, 0.8986608676234962, 0.8980461131820265, 0.897857393429826, 0.8976514823846418, 0.8971597892846933, 0.8968202252017837, 0.896010926435072, 0.8939120779902925, 0.8938877505927909, 0.8935600985024458, 0.8934094173937303, 0.8928923190406759, 0.892384427020994, 0.8910105615174445, 0.890806443235996, 0.890371

 99%|█████████▉| 99/100 [00:01<00:00, 89.32it/s][0.7843566293198136, 0.7724002507123898, 0.7676408830108097, 0.748886311423223, 0.7418867267887965, 0.7351914686841801, 0.7313154386647791, 0.7279952753987481, 0.7164221141104069, 0.7126908323023745, 0.7009394926061773, 0.6985348473249141, 0.6929204504844226, 0.6926981731730655, 0.6880754740517068, 0.6842119745536241, 0.6811850523998347, 0.6766205305578612, 0.676274847115059, 0.6756841400022167, 0.6755116176562588, 0.6746697676139084, 0.6705334186766809, 0.6649179156808985, 0.6623125123610356, 0.6601078418830909, 0.6594149209426489, 0.6508456173890499, 0.6499947743068374, 0.6482972265580405, 0.6481267203296264, 0.6430465992867211, 0.6415301874770206, 0.6397601715156429, 0.6389868996490076, 0.6383263614131884, 0.6369067440465638, 0.6344344617373603, 0.6338557818295479, 0.6337217101078, 0.6332091007038594, 0.632625446114532, 0.6307491640074628, 0.6306305460842057, 0.6300242597151582, 0.6289232822370813, 0.6261221351731379, 0.623342733338028

100%|██████████| 100/100 [00:01<00:00, 83.05it/s]
ModelEvaluation(mapk=0.7394630952380952)


In [38]:
evaluations = k_fold_cross_validation(
  train_df,
  k=CROSS_VALIDATION_K,
  train_model_fn=train_word2vec,
  preprocess_val_fn=None,
  build_ranking_fn=lambda word2vec: lambda user_id, songs_df: word2vec_rank_by_user(word2vec.wv, user_id, songs_df))
mapks = list(map(lambda evaluation: evaluation.mapk, evaluations))
np.mean(mapks)

****************************************************************************************************
Taking split number 0 as the validation set...
Train indexes: 5901934
Val indexes: 1475484
Users: 30548
Building sessions
Building vocabulary
Training word2vec
Word2Vec(vocab=88866, size=32, alpha=0.03)
Building user embeddings
5901934it [05:22, 18307.65it/s]
Evaluating the model
  0%|          | 0/100 [00:00<?, ?it/s][nan, nan, 0.7880005389308355, 0.7101731311951163, nan, nan, nan, nan, 0.7748911874260385, 0.7479348832680403, 0.7448554576789017, nan, 0.7472644274770357, 0.7413766472868255, 0.6703330586545072, 0.6559112264128524, 0.6323498022860515, 0.6188911196773353, 0.6166856151001509, 0.6100725705763418, 0.6098558428744367, nan, nan, 0.6362692287818092, 0.6309063742526874, 0.6000691412550989, 0.5998039358352925, 0.5900736969185105, 0.5615207931363457, 0.5556093051048546, 0.5442807337565491, 0.5439947761619812, 0.5429990418947019, 0.539857906959352, 0.5318945453780919, 0.510189699790

 24%|██▍       | 24/100 [00:00<00:00, 230.88it/s][0.8453213973222007, nan, 0.8502536772324023, 0.8276510051541799, 0.8124952704771639, 0.7330068474159356, nan, 0.7956572301822575, 0.7897052455945858, 0.785900728648725, 0.785820138186086, 0.7658809855294935, nan, 0.7773287380663404, 0.7669200418637753, 0.7463668503551856, 0.7456346362276328, 0.7377801047385141, 0.7297357123813183, 0.7231038636526314, 0.7125520740627298, 0.7025689862072023, 0.6975917797359881, 0.6893874650040588, 0.6834129906609983, 0.6722686973998063, 0.6677533883418956, 0.6669118745109355, 0.662752451762519, 0.6594756186686698, 0.6576136441850455, 0.6550089113137881, 0.6533872339734997, 0.6524070433550161, 0.6340558730484478, 0.6321532883295327, 0.6276634253662858, 0.6187437880026727, 0.6182491910008747, 0.6117934912354741, 0.5983328488553125, 0.5960805549379208, 0.5937054759047137, 0.5908508317885073, 0.5899801941993595, 0.5645525615327907, 0.5584085136064738, 0.5578448794282899, 0.5537054606033418, 0.5523509767736453

[0.9076116153433352, 0.906428161741543, 0.8981238493209486, 0.8939239596955125, 0.8824748049159493, 0.8787618350251006, 0.8750389052542447, 0.8616971205236442, 0.8604694927030331, 0.8598346941625159, 0.8564657971403105, 0.8476858859111216, 0.8341541648463081, 0.8265350295826035, 0.8247413322390683, 0.8176221411637604, 0.8154901669937724, 0.8154508897271139, 0.8154034796686493, 0.8128675863949552, 0.8125161068088369, 0.8114407775118592, 0.8104463557582171, 0.8103868824475418, 0.7978684707813062, 0.794440246288384, 0.7938772866046556, 0.793566204174808, 0.7907746222056466, 0.7852174436123456, 0.7844396823023765, 0.7841123276383594, 0.7805587302060346, 0.7801695768591552, 0.7796554959645722, 0.77959690865507, 0.7724875147072335, 0.768873474479383, 0.768555602365889, 0.7673327854143187, 0.7636843967336586, 0.7615892992102795, 0.7595412053103822, 0.753633219527769, 0.7529666330030035, 0.7475168415519867, 0.7398396743869022, 0.7380057920778145, 0.7361159724527863, 0.7348705776636871, 0.73417

[0.8598957444607794, nan, nan, 0.8518932299571412, 0.8496718111376714, 0.8492604394374708, 0.8489218455332282, 0.8454014184135502, 0.8448682956428022, 0.8283772564041362, 0.8270987712897498, 0.8211809368972315, 0.8210606904713409, 0.8157709423305148, 0.7670068630065758, 0.7615852420253714, 0.7582703724574268, 0.7566174571475044, 0.7479081186619484, 0.7421373908716593, 0.7362807627639746, 0.7309695586883707, 0.7259796082102226, 0.7240255294771981, 0.7215691458387705, 0.7040955883405744, 0.6787685143457133, 0.6740702978790636, 0.673831954938764, 0.6736827332628798, 0.6668265562310691, 0.665418108799079, 0.6646907934254159, 0.6637599306371168, 0.6543020280407209, 0.6504817169919893, 0.6405992892247113, 0.6356498717873944, 0.593731655176638, 0.5934635883011329, 0.5890978377248498, 0.5720727172517657, 0.5358469755454143, 0.533734435308102, 0.5085010086947352, 0.4808668296764208, 0.45507545626050405, 0.4054836619218889, nan, 0.8437485849258523, 0.8034445870207323, 0.7995345665884402, 0.79653

 74%|███████▍  | 74/100 [00:00<00:00, 238.29it/s][0.8493581066779426, 0.8398373791305789, 0.8285484500680954, 0.7284456839356662, 0.7268019222169474, 0.7243716208686191, 0.6990408088947382, 0.6894067988666509, 0.6796065740570281, 0.6756958019338168, 0.673084767570446, 0.6714986872181427, 0.6675430672475888, 0.6626178894500835, 0.6474051931177781, 0.6327164112077679, 0.6082115666615794, 0.604539031769092, 0.594095165438788, 0.5290124946607184, 0.5152476731338869, 0.4982494233925669, 0.4673920342986929]
[0.840575373472402, 0.8267000438336212, 0.8178784029761673, 0.8160134541442002, 0.8149542672097733, 0.8124185280197831, 0.8104045831068876, nan, nan, 0.8151213262040767, 0.8095319006186821, 0.8081480855851026, 0.8049715223901116, 0.8044876992134707, 0.8044109997343801, 0.8026116822382966, 0.8007765413549977, 0.8005870552338628, 0.7902764438371446, 0.7886915790559741, 0.7854356967857028, 0.7837998000995652, 0.7828685264995887, 0.7793044885935495, 0.7788511270174581, 0.776392965313994, 0.77

...

[nan, 0.7205100926156559, 0.7158447563276011, 0.7080930538858677, 0.7064911238250287, 0.7020395037697873, 0.7001524461709508, 0.6981431795789999, 0.6969104660090479, 0.6960465379568326, 0.6953422185708651, 0.6949431581321264, 0.6879003590894409, 0.687334432810566, 0.6868736260055921, 0.6833455624299376, 0.6813123438851199, 0.6769324110423124, 0.6754203617633592, 0.6736587879561962, 0.6723523168124094, 0.6714198260645168, 0.6651116421585124, 0.6647660907108504, 0.6599779064503044, 0.6581803839263736, 0.6553089941230902, 0.6512346151422664, 0.6508019814068217, 0.649463582223339, 0.6467535031181195, 0.6431548527557498, 0.6418763620623342, 0.6394303833676069, 0.633544399046844, 0.6318729496811278, 0.6305761110744893, 0.6304885228090847, 0.6298726501552181, 0.6278653686930451, 0.6230821931289323, 0.6212803653295413, 0.6209653767738261, 0.6174066623573103, 0.6173998835700385, 0.6159296457709599, 0.6106106771956203, 0.6097785453791617, 0.6027231946578395, 0.6023565561708141, 0.590546238888136

[nan, nan, nan, 0.8121675565155841, 0.7944358736674868, 0.7659247882478092, 0.7576266677056643, 0.7492484570135022, 0.7483670045137306, 0.7407676608891249, 0.7220953791754403, 0.720559359186495, 0.7163742004491472, 0.6996214638125405, 0.6984344578889924, 0.6971149917404179, 0.6843375781218579, 0.6554473301928195, 0.6533745188315444, 0.6442695225420324, 0.6427528350506483, 0.6391240770093746, 0.6324799931231978, 0.597230127098298, 0.5885176773671721, 0.5747486016595971, 0.566237706831646, 0.558073540232478, 0.5547198055654895, 0.5491982114886106, 0.5389612296565021, 0.5360028839150813, 0.5357975556712544, 0.5351149571063023, 0.5329584157046483, 0.5298660401976223, 0.5247000805436584, 0.5234074873244056, 0.5199622186833137, 0.5095803663440239, 0.5038175296641269, 0.450114818285353, 0.4469049571597042, 0.4021666073944868, nan, 0.861244442798762, 0.8556430351166123, 0.821771693216604, 0.8137149361937895, 0.8072194618310295, 0.7488481168885738, 0.7393771075432154, 0.7326026313870629, 0.7169

[0.8311368828078248, 0.7910228474504166, 0.7891911385776411, 0.7729156018410451, 0.7725577677811655, nan, nan, 0.8019585980097746, 0.7693521728941188, 0.7690256977749819, 0.7548445114748321, 0.7446386059588458, 0.7434746189726005, 0.7352575273814992, 0.7216746790494872, 0.7110650187161687, 0.7104466657179358, 0.7085271602156674, 0.7072947694042748, 0.6951949707920358, 0.6949229261208484, 0.6946311111405689, 0.6862091403754799, 0.6858384518907156, 0.6773142115703877, 0.6760490410221097, 0.6747505048361082, 0.6656335830973479, 0.6597058097178031, 0.6512741102154219, 0.6496642688460976, 0.6444260783014085, 0.6384941083163684, 0.6346460878810253, 0.6298337347460735, 0.6212318054987239, 0.6144321139829826, 0.6138110751657302, 0.6007825398756074, 0.5945745188884017, 0.5940997271184999, 0.5933211990680983, 0.5926682063035389, 0.5922478372699062, 0.5917125944073764, 0.5901534465173315, 0.5894715318403163, 0.5838747826937217, 0.5671209422296074, 0.5440947894987579, 0.5176430715882991, 0.4980701

100%|██████████| 100/100 [00:00<00:00, 258.46it/s]
ModelEvaluation(mapk=0.5168065192743764)


0.5061806415343915

## Word2Vec + GBDT

In [39]:
def get_user_song_similarity(wv, row):
  global user_embeddings
  user_id = row['msno']
  song_id = row['song_id']
  user_embedding = user_embeddings[user_id]
  song_embedding = wv[song_id] if song_id in wv else np.zeros(WORD2VEC_EMBEDDING_SIZE)
  return word2vec_similarity(user_embedding, song_embedding)

def extend_data_with_word2vec(word2vec, X):
  wv = word2vec.wv
  X = X.copy()
  X['w2v_sim'] = X.apply(lambda row: get_user_song_similarity(word2vec.wv, row), axis=1)
  return X

def train_word2vec_lightgbm(X_train, y_train):
  word2vec = train_word2vec(X_train, y_train)
  X_train = extend_data_with_word2vec(word2vec, X_train)
  model = train_lightgbm(X_train, y_train)
  return word2vec, model

In [40]:
evaluations = k_fold_cross_validation(
  train_df,
  k=CROSS_VALIDATION_K,
  train_model_fn=train_word2vec_lightgbm,
  preprocess_val_fn=lambda models, X: extend_data_with_word2vec(models[0], X),
  build_ranking_fn=lambda models: lambda user_id, songs_df: lightgbm_rank_by_user(models[1], user_id, songs_df))
mapks = list(map(lambda evaluation: evaluation.mapk, evaluations))
np.mean(mapks)

****************************************************************************************************
Taking split number 0 as the validation set...
Train indexes: 5901934
Val indexes: 1475484
Users: 30548
Building sessions
Building vocabulary
Training word2vec
Word2Vec(vocab=88866, size=32, alpha=0.03)
Building user embeddings
5901934it [05:18, 18507.99it/s]
Training the model
[10]	training's auc: 0.763694
[20]	training's auc: 0.774459
[30]	training's auc: 0.77925
[40]	training's auc: 0.783557
[50]	training's auc: 0.786928
Evaluating the model
100%|██████████| 100/100 [03:48<00:00,  2.28s/it]
ModelEvaluation(mapk=0.6797367929579239)
****************************************************************************************************
Taking split number 1 as the validation set...
Train indexes: 5901934
Val indexes: 1475484
Users: 30543
Building sessions
Building vocabulary
Training word2vec
Word2Vec(vocab=88921, size=32, alpha=0.03)
Building user embeddings
5901934it [05:19, 18499.12it/s

0.6846374140211641

In [41]:
X_train = train_df.drop(['target'], axis=1)
y_train = train_df['target'].values
word2vec, model = train_word2vec_lightgbm(X_train, y_train)
X_train = extend_data_with_word2vec(word2vec, X_train)
evaluation = evaluate_model_ranking_fn(
  lambda user_id, songs_df: lightgbm_rank_by_user(model, user_id, songs_df),
  X_train,
  y_train)

Users: 30755
Building sessions
Building vocabulary
Training word2vec
Word2Vec(vocab=101648, size=32, alpha=0.03)
Building user embeddings
7377418it [06:41, 18375.53it/s]
Training the model
[10]	training's auc: 0.761852
[20]	training's auc: 0.771767
[30]	training's auc: 0.776691
[40]	training's auc: 0.780748
[50]	training's auc: 0.784544
Evaluating the model
100%|██████████| 100/100 [03:47<00:00,  2.27s/it]
ModelEvaluation(mapk=0.8461059523809525)


In [42]:
np.random.seed(SEED)
explain_indexes = np.random.choice(X_train.index, EXPLAIN_FROM_SAMPLES, replace=False)
X_explain = X_train.iloc[explain_indexes]

In [43]:
explainer = shap.TreeExplainer(model, feature_perturbation='tree_path_dependent')
shap_values = explainer.shap_values(X_explain)

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [44]:
index = 0
shap.initjs()
shap.force_plot(
  explainer.expected_value[0],
  shap_values[0][index,:],
  X_explain.iloc[index,:])

In [45]:
X_test = test_df.drop(['id'], axis=1)
X_test = extend_data_with_word2vec(word2vec, X_test)
test_ids = test_df['id'].values

p_test = model.predict(X_test)

submission_df = pd.DataFrame()
submission_df['id'] = test_ids
submission_df['target'] = p_test
submission_df.to_csv('lightgbm_word2vec_submission_df.csv', index=False, float_format='%.6f')

In [46]:
# TOOD(niksaz): Update it to print the time taken to run the notebook.

from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 04:10:23
